In [ ]:
import os

import numpy as np
import pandas as pd
import psutil
from resolve_path import ajuste_path, read_input

In [ ]:
pathUtil = ajuste_path('data/util/')
df = pd.read_csv(pathUtil + 'dataset_associado.csv', encoding='utf-8')

In [ ]:
# Passar a data para datetime
df['data_inicio'] = pd.to_datetime(df['data_inicio'])
df['data_fim'] = pd.to_datetime(df['data_fim'])

In [ ]:
# criação da coluna de trabalho por mes
df["meses"] = (
    (df["data_fim"].dt.year - df["data_inicio"].dt.year) * 12
    + (df["data_fim"].dt.month - df["data_inicio"].dt.month)
) + 1
df["hh_por_mes"] = df["hh"] / df["meses"]

EXPANSÃO DE OS POR MES

In [ ]:
def expande_df(df):
    # Gerar o espaço de tempo
    espaco_de_tempo = pd.date_range(
        start='2020-01-01', end=pd.Timestamp.now(), freq="MS")
    espaco_de_tempo = pd.DataFrame(
        {'merge': [1]*len(espaco_de_tempo), 'ano_mes': espaco_de_tempo})
    # Marcar operações que duram mais de um mês
    df.loc[df['meses'] > 1, 'dura_mais_de_mes'] = 1

    # Dropando colunas desnecessárias
    colunas_a_dropar = ["hh"]
    df = df.drop(columns=colunas_a_dropar)

    # Expansão das operações que duram mais de um mês
    df_expandido = pd.merge(df, espaco_de_tempo, left_on='dura_mais_de_mes',
                            right_on='merge', how='left').drop(columns=['merge'])
    return df_expandido


df_expandido = expande_df(df)

colunas_tipo_atividade = ["txt.breve_operacao", "denominacao_tam"]
# Tratamento das colunas de data para garantir o correto agrupamento
df_expandido['dura_mais_de_mes'] = df_expandido['dura_mais_de_mes'].fillna(0)
df_expandido.loc[df_expandido['dura_mais_de_mes']
                 == 0, 'ano_mes'] = df_expandido['data_fim']
df_expandido['ano_mes'] = df_expandido['ano_mes'].dt.to_period('M')
df_expandido['data_inicio'] = df_expandido['data_inicio'].dt.to_period('M')
df_expandido['data_fim'] = df_expandido['data_fim'].dt.to_period('M')

df_expandido['data_acidente'] = pd.to_datetime(df_expandido['data_acidente'])
df_expandido['data_acidente'] = df_expandido['data_acidente'].dt.to_period('M')


# Eliminar linhas onde a operação não estava ativa
df_expandido = df_expandido[(df_expandido['ano_mes'] >= df_expandido['data_inicio']) &
                            (df_expandido['ano_mes'] <= df_expandido['data_fim'])]


# separar colunas de acidente apenas
lista_colunas_acidentes = df_expandido.columns[df_expandido.columns.str.contains(
    'acidente')].tolist()

# Colocar NAN nas colunas da acidente para as linhas em a data de acidente nao bate com a data da operação
df_expandido.loc[(df_expandido['data_acidente'] != pd.NA) & (
    df_expandido['ano_mes'] != df_expandido['data_acidente']), lista_colunas_acidentes] = pd.NA


# Para cada categoria de atividade, criar uma nova coluna para armazenar o HH dessa categoria
for coluna_tipo_atividade in colunas_tipo_atividade:
    for atividade in df_expandido[coluna_tipo_atividade].unique():
        if "hh_de_" + atividade in df_expandido.columns:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "hh_de_" + atividade + "_de_" + coluna_tipo_atividade] = df_expandido["hh_por_mes"]
        else:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "hh_de_" + atividade] = df_expandido["hh_por_mes"]

df_expandido.drop(columns=['txt.breve_operacao',
                  'tam', 'denominacao_tam'], inplace=True)

CRIAÇÃO DE DICIONÁRIO

In [ ]:
# Define dicionario, indicando qual operação será efetuada sobre quais colunas durante o agrupamento

# Retirando as colunas que são as variáveis do groupby da lista do dicionário
colunas = list(df_expandido.columns)
colunas.remove("no_pessoal")
colunas.remove("local_de_instalacao")
colunas.remove("ano_mes")

# # Separando colunas de hh
colunas_hh = [
    col for col in colunas if col.startswith('hh')]

# # Pegando apenas as colunas que nao sao de hh
colunas_sem_hh = [col for col in colunas if not col.startswith('hh')]

# # Definindo operacoes que serao aplicadas sobre as colunas
dicionario_hh = {col: 'sum' for col in colunas_hh}
dicionario_hh.update({col: 'first' for col in colunas_sem_hh})

AGRUPAMENTO POR MES ANO LOCAL ID

In [ ]:
# Agrupando por ano_mes, local e no_pessoa, igualmente operando sobre as outras colunas
df_agrupado = df_expandido.groupby(
    ['ano_mes', 'local_de_instalacao', 'no_pessoal']).agg(dicionario_hh).reset_index()
df_agrupado.rename(columns={'hh_por_mes': 'hh_total'}, inplace=True)

# Filtra para considerar apenas registros a partir de 2020-01
df_agrupado = df_agrupado[df_agrupado['ano_mes'] >= '2020-01']

ACRESCENTAR COLUNA PARA INDICAR SE TEVE OU NÃO ACIDENTE

In [ ]:
df_agrupado['acidente'] = pd.NA

df_agrupado.loc[df_agrupado['id_acidente'].notna(), 'acidente'] = 1
df_agrupado.loc[df_agrupado['id_acidente'].isna(), 'acidente'] = 0

EXPORTANDO DATASET

In [ ]:
df_agrupado.to_csv(pathUtil + "dataset_treinamento.csv",
                   encoding='utf-8', index=False)